In [3]:
import pandas as pd

df = pd.read_excel(r"D:\downloads\Name model 1.xlsx", sheet_name="distribution", header=None)

segment_dfs = {}
i = 0
while i < len(df):
    row = df.iloc[i]
    non_empty_count = row.notna().sum()

    # A segment header = row with exactly 4 non-empty cells
    # followed by a row that is not entirely empty
    if non_empty_count == 4 and not df.iloc[i+1].isna().all():
        header = row
        # Find where this segment ends (next completely empty row)
        j = i + 1
        while j < len(df) and not df.iloc[j].isna().all():
            j += 1

        # Slice out this segment’s data
        segment_data = df.iloc[i+1:j].dropna(how='all').reset_index(drop=True)
        segment_data.columns = header

        # Use first column value of header row as segment name
        segment_name = str(header.iloc[0])
        segment_dfs[segment_name] = segment_data

        # Jump index to the end of this segment
        i = j
    else:
        i += 1

In [12]:
import pandas as pd
import numpy as np
from pptx import Presentation
from pptx.util import Cm, Pt
from pptx.chart.data import CategoryChartData
from pptx.enum.chart import XL_CHART_TYPE, XL_LEGEND_POSITION
from pptx.dml.color import RGBColor
from pptx.enum.dml import MSO_LINE_DASH_STYLE


# ----------- Number Formatting Helper ------------
def format_number(val):
    """Format number into compact form (k/M suffix)."""
    if val is None:
        return ""
    try:
        v = float(val)
    except:
        return ""
    if v >= 1_000_000:
        return f"{v/1_000_000:.1f}M"
    elif v >= 1000:
        return f"{v/1000:.0f}k"
    else:
        return str(int(v))


# ----------- Custom Labels (font only) ------------
def apply_custom_labels(series, values, is_percent=False):
    """
    Apply formatted labels with compact numbers or percentages.
    """
    for point, val in zip(series.points, values):
        lbl = point.data_label
        tf = lbl.text_frame
        tf.clear()

        # Add formatted text
        p = tf.paragraphs[0]
        run = p.add_run()
        if is_percent:
            try:
                run.text = f"{float(val):.2%}" if val is not None else ""
            except:
                run.text = ""
        else:
            run.text = format_number(val)

        # Font style
        run.font.size = Pt(8)
        run.font.bold = True


# ----------- Clean Values Helper ------------
def clean_values(values):
    """Ensure no NaN/Inf goes into PowerPoint chart data."""
    numeric = pd.to_numeric(values, errors="coerce")  # force numeric
    cleaned = []
    for v in numeric:
        if pd.isna(v) or np.isinf(v):
            cleaned.append(None)
        else:
            cleaned.append(float(v))
    return cleaned


# ----------- Create PowerPoint ------------
prs = Presentation()
blank_layout = prs.slide_layouts[6]  # fully blank slide

for idx, (segment_name, seg_df) in enumerate(segment_dfs.items()):
    # Add a new slide every 4 charts
    if idx % 4 == 0:
        slide = prs.slides.add_slide(blank_layout)

    # Chart positions (4 per slide)
    positions = [
        (Cm(1), Cm(1)),
        (Cm(16), Cm(1)),
        (Cm(1), Cm(11)),
        (Cm(16), Cm(11)),
    ]
    left, top = positions[idx % 4]
    width, height = Cm(11.48), Cm(7.62)

    # --- Build chart data ---
    chart_data = CategoryChartData()
    chart_data.categories = list(seg_df.iloc[:, 0])  # first col = categories
    chart_data.add_series("Total Count", clean_values(seg_df["Total_count"]))
    chart_data.add_series("Cancel Rate", clean_values(seg_df["Cancel_rate"]))
    chart_data.add_series("Average Cancel Rate", clean_values(seg_df["Average_cancel_rate"]))

    # --- Create chart ---
    chart = slide.shapes.add_chart(
        XL_CHART_TYPE.COLUMN_CLUSTERED, left, top, width, height, chart_data
    ).chart

    # Remove gridlines
    chart.value_axis.has_major_gridlines = False

    # --- Format Total Count (bar) ---
    bar_series = chart.series[0]
    bar_series.chart_type = XL_CHART_TYPE.COLUMN_CLUSTERED
    bar_values = clean_values(seg_df["Total_count"])
    apply_custom_labels(bar_series, bar_values, is_percent=False)

    # --- Format Cancel Rate (orange line) ---
    line_series = chart.series[1]
    line_series.chart_type = XL_CHART_TYPE.LINE
    line_series.format.line.color.rgb = RGBColor(255, 165, 0)
    line_values = clean_values(seg_df["Cancel_rate"])
    apply_custom_labels(line_series, line_values, is_percent=True)

    # --- Format Average Cancel Rate (green dotted line, no labels) ---
    avg_series = chart.series[2]
    avg_series.chart_type = XL_CHART_TYPE.LINE
    avg_series.format.line.color.rgb = RGBColor(0, 128, 0)
    avg_series.format.line.dash_style = MSO_LINE_DASH_STYLE.DASH_DOT
    avg_series.format.line.width = Cm(0.1)
    avg_series.data_labels.show_value = False  # no labels

    # --- Chart title ---
    chart.has_title = True
    chart.chart_title.text_frame.text = f"{segment_name}"
    title_font = chart.chart_title.text_frame.paragraphs[0].font
    title_font.bold = True
    title_font.size = Pt(10)

    # --- Legend ---
    chart.has_legend = True
    chart.legend.position = XL_LEGEND_POSITION.BOTTOM
    # for legend_entry in chart.legend.entries:
    #     for paragraph in legend_entry.text_frame.paragraphs:
    #         for run in paragraph.runs:
    #             run.font.size = Pt(9)

    # --- Axis font size ---
    chart.category_axis.tick_labels.font.size = Pt(9)
    chart.value_axis.tick_labels.number_format = '#,##0,"k"'
    chart.value_axis.tick_labels.font.size = Pt(9)


# ----------- Save presentation ------------
output_ppt = "segments_charts_editable.pptx"
prs.save(output_ppt)
print(f"✅ PowerPoint saved as {output_ppt} with editable charts")

✅ PowerPoint saved as segments_charts_editable.pptx with editable charts
